In [12]:
#from EEGP import EEGPreprocessor
from EGGP_prod import EEGP_prod_processor
import pandas as pd
import numpy as np
from sklearn.utils import resample
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pywt

In [30]:
df = pd.read_csv("/Users/charleshajjar/Documents/DataAnalyse/data/CaptureSamples/charles_5m-2s-2.csv")
df = df.iloc[:300]

In [31]:
df

,EEG 1,EEG 2,EEG 3,EEG 4,EEG 5,EEG 6,EEG 7,EEG 8,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Battery Level,Counter,Validation Indicator,Display,Direction,Timestamp
0,103396.00,53543.00,-28492.400,36144.200,165513.00,-4847.91,750000.00,-750000.00,0.040772,0.880859,-0.308350,0.000000,0.00000,0.000000,80.0,1,1,Text,Left,3580
1,-18119.60,73768.30,0.000,150.383,94810.80,-4486.35,423827.00,-750000.00,0.034424,0.896484,-0.323486,0.000000,0.00000,0.000000,80.0,2,1,Text,Left,6221
2,-158569.00,52005.30,0.000,-431.836,54353.50,-5413.32,167753.00,-750000.00,0.037353,0.909180,-0.323242,0.000000,0.00000,0.000000,80.0,3,1,Text,Left,8585
3,-124263.00,39013.50,-454.277,-712.127,41557.10,-9218.22,92342.80,-270984.00,0.043213,0.918213,-0.324707,0.000000,0.00000,0.000000,80.0,4,1,Text,Left,11063
4,-16641.00,30838.80,-4731.770,2422.570,31535.50,-8180.38,60563.90,-26231.20,0.047119,0.924561,-0.325195,-0.152588,-0.12207,-0.152588,80.0,5,1,Text,Left,13727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,80.0,296,0,Text,Left,1768926
296,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,80.0,297,0,Text,Left,1769012
297,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,80.0,298,0,Text,Left,1769093
298,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,80.0,299,0,Text,Left,1769178


In [14]:
df = df[["EEG 1", "EEG 2", "EEG 3", "EEG 4", "EEG 5", "EEG 6", "EEG 7", "EEG 8",
                 "Accelerometer X","Accelerometer Y", "Accelerometer Z", "Gyroscope X",
                 "Gyroscope Y", "Gyroscope Z", "Timestamp"]]

In [15]:
df

,EEG 1,EEG 2,EEG 3,EEG 4,EEG 5,EEG 6,EEG 7,EEG 8,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Timestamp
0,103396.00,53543.00,-28492.400,36144.200,165513.00,-4847.91,750000.00,-750000.00,0.040772,0.880859,-0.308350,0.000000,0.00000,0.000000,3580
1,-18119.60,73768.30,0.000,150.383,94810.80,-4486.35,423827.00,-750000.00,0.034424,0.896484,-0.323486,0.000000,0.00000,0.000000,6221
2,-158569.00,52005.30,0.000,-431.836,54353.50,-5413.32,167753.00,-750000.00,0.037353,0.909180,-0.323242,0.000000,0.00000,0.000000,8585
3,-124263.00,39013.50,-454.277,-712.127,41557.10,-9218.22,92342.80,-270984.00,0.043213,0.918213,-0.324707,0.000000,0.00000,0.000000,11063
4,-16641.00,30838.80,-4731.770,2422.570,31535.50,-8180.38,60563.90,-26231.20,0.047119,0.924561,-0.325195,-0.152588,-0.12207,-0.152588,13727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,1768926
296,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,1769012
297,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,1769093
298,3659.61,3635.91,3634.570,3636.990,3648.79,3672.84,3637.17,3656.92,0.059082,0.918945,-0.335938,-1.495360,2.68555,2.075200,1769178


In [16]:
preprocessor = EEGP_prod_processor(df)
filtered_df = preprocessor.apply_butter_filter()
results_df = preprocessor.detect_threshold_changes()
filtered_df = preprocessor.convert_timestamp()
filtered_df = preprocessor.label_thinking_periods(results_df)
transformed_data_df = preprocessor.transform_data(filtered_df)
calculated_p_value_df = preprocessor.calculate_p_value(filtered_df)
p_value_df = filtered_df.assign(p_value=calculated_p_value_df['p_value'])
fdr_corrected_df = preprocessor.fdr_correction(p_value_df)
dwt_grouped_sequences_df = preprocessor.dwt_grouped_sequences_prod(fdr_corrected_df, ["Accelerometer X", "Accelerometer Y",
                      "Accelerometer Z", "Gyroscope X", "Gyroscope Y", "Gyroscope Z"])
df_inter = dwt_grouped_sequences_df
df_final = preprocessor.label_encode_target(df_inter)

/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"EEG Column": column, "Start Time": start_time, "End Time": end_time},
/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"EEG Column": column, "Start Time": start_time, "End Time": end_time},
/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"EEG Column": column, "Start Time": start_time, "End Time": end_time},
/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:102: FutureWarning: The frame.append method is deprecated and will 

In [17]:
dwt_grouped_sequences_df

,Relative Energy,Target,p_value_fdr,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Mean Spectral Accel X,Mean Spectral Accel Y,Mean Spectral Accel Z,Mean Spectral Gyro X,Mean Spectral Gyro Y
0,37.952802,0,0.599586,0.053201,0.924798,-0.332757,-0.653253,2.504210,1.424153,1.675095,3.690443,27.017388,9.356745,23.394772
1,15.419194,1,0.599586,0.046956,0.926677,-0.325765,-1.475015,-0.803630,-0.040690,0.502898,1.098420,1.289513,0.261939,1.389926
2,43.991315,0,0.396129,0.042572,0.923498,-0.325893,-0.817152,4.474595,-0.177720,1.788236,4.090322,33.825659,11.681456,29.286424
3,0.800380,1,0.396129,0.034424,0.896484,-0.323486,0.000000,0.000000,0.000000,0.289230,0.753056,0.167155,0.039821,0.077952
4,33.132119,0,0.396129,0.052436,0.923042,-0.333550,-0.111898,1.203749,2.261691,0.672707,1.832408,1.435746,0.388637,1.268992
5,0.793933,1,0.396129,0.040772,0.880859,-0.308350,0.000000,0.000000,0.000000,0.281829,0.732349,0.163962,0.038907,0.074305
6,28.009960,0,0.396129,0.032227,0.911621,-0.323975,0.030518,4.394530,-0.457764,0.432130,0.770176,3.397465,1.208417,2.610817
7,0.841672,1,0.396129,0.039307,0.912191,-0.323730,0.000000,0.000000,0.000000,0.440443,1.145550,0.255697,0.060797,0.117193
8,28.861027,0,0.363635,0.049434,0.922422,-0.331709,-0.113524,0.710450,1.843261,0.746361,2.319749,1.585653,0.538186,1.282461
9,210.676246,1,0.363635,0.043701,0.932129,-0.331299,-0.732422,1.617430,7.080080,0.449927,0.601802,0.160836,0.001718,0.264078


In [18]:
df_final

,Relative Energy,Target,p_value_fdr,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Mean Spectral Accel X,Mean Spectral Accel Y,Mean Spectral Accel Z,Mean Spectral Gyro X,Mean Spectral Gyro Y
0,37.952802,0,0.599586,0.053201,0.924798,-0.332757,-0.653253,2.504210,1.424153,1.675095,3.690443,27.017388,9.356745,23.394772
1,15.419194,1,0.599586,0.046956,0.926677,-0.325765,-1.475015,-0.803630,-0.040690,0.502898,1.098420,1.289513,0.261939,1.389926
2,43.991315,0,0.396129,0.042572,0.923498,-0.325893,-0.817152,4.474595,-0.177720,1.788236,4.090322,33.825659,11.681456,29.286424
3,0.800380,1,0.396129,0.034424,0.896484,-0.323486,0.000000,0.000000,0.000000,0.289230,0.753056,0.167155,0.039821,0.077952
4,33.132119,0,0.396129,0.052436,0.923042,-0.333550,-0.111898,1.203749,2.261691,0.672707,1.832408,1.435746,0.388637,1.268992
5,0.793933,1,0.396129,0.040772,0.880859,-0.308350,0.000000,0.000000,0.000000,0.281829,0.732349,0.163962,0.038907,0.074305
6,28.009960,0,0.396129,0.032227,0.911621,-0.323975,0.030518,4.394530,-0.457764,0.432130,0.770176,3.397465,1.208417,2.610817
7,0.841672,1,0.396129,0.039307,0.912191,-0.323730,0.000000,0.000000,0.000000,0.440443,1.145550,0.255697,0.060797,0.117193
8,28.861027,0,0.363635,0.049434,0.922422,-0.331709,-0.113524,0.710450,1.843261,0.746361,2.319749,1.585653,0.538186,1.282461
9,210.676246,1,0.363635,0.043701,0.932129,-0.331299,-0.732422,1.617430,7.080080,0.449927,0.601802,0.160836,0.001718,0.264078


In [19]:
#df_final = preprocessor.label_encode_target(dwt_grouped_sequences_df)

In [20]:
df_final.Target.value_counts()

0    8
1    8
Name: Target, dtype: int64

In [21]:
# Triez les données selon le critère "Target" et "Relative Energy"
df_final = df_final.loc[df_final["Relative Energy"] >= 18]
df_final = df_final.sort_values(by=["Target", "Relative Energy"], ascending=[True, False])
df_final = df_final.reset_index(drop=True)

In [26]:
X = df_final.values

In [27]:
from keras.models import load_model
model = load_model('modelprocess1')

In [28]:
val_predictions = model.predict(X)
val_classes = val_predictions.argmax(axis=1)

1/1 [==============================] - 0s 230ms/step


In [29]:
val_classes

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 3])